#### Merge & Serve

In [ ]:
from src import PeftInferencer

# Unfortunately, full-precision LoRA Adaptor model is not doing Too Well now
adaptor_id = "Ksgk-fy/phillipine_customer_v3.5_intro_phase"
f = PeftInferencer(adaptor_id, use_quant=False)
merge_model = f.model.merge_and_unload()
merge_model.push_to_hub("Ksgk-fy/ecoach_philippine_v6_intro_merge") # Merge and Push to Huggingface -- just so that we could serve with vLLM easily
f.tokenizer.push_to_hub("Ksgk-fy/ecoach_philippine_v6_intro_merge")

In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from prometheus_eval.vllm import VLLM
from prometheus_eval import PrometheusEval
from prometheus_eval.prompts import ABSOLUTE_PROMPT, SCORE_RUBRIC_TEMPLATE

# model_id = "prometheus-eval/prometheus-bgb-8x7b-v2.0"
model_id = "prometheus-eval/prometheus-7b-v2.0"
model = VLLM(model=model_id)
judge = PrometheusEval(model=model, absolute_grade_template=ABSOLUTE_PROMPT)

In [19]:
# Prompt to ellicit self-reflection mechanism 
# Spawn & Termination of Threading is quite useful indeed.


from src.models.openai_v2 import OpenAIModel

m = OpenAIModel(model_name="gpt-3.5-turbo")

# Iterative Placement of Bad Cases & Rationale 
bad_responses = ["Response: How can I assist you? Rationale: Maria is a customer, she is not supposed to pay attention to assisting the sales agent."]
good_responses = ["Response: Hey, how is it going? Rationale: Casual and conversational response is good."]

# Prompt the model to reflect and self-judge, based on the good & bad responses
# Context on which it conducts the reflection will be different than when it generates the response



In [ ]:
prompts = [
    "Hi! Are you Maria?",
    "I know that feeling, Manila traffic can be quite a challenge! Have you been in this area before?",
    "Whether in Manila recently have been very hot, how is life recently?",
    "Hi! Are you familiar with this part of the city?",
    "I heard there's a great café nearby. Have you tried it?",
    "Manila is such a vibrant place! Do you have any favorite spots here?",
    "The local markets here are amazing. Have you visited any recently?",
    "It's always busy around here! How do you usually spend your time in Manila?",
    "Do you know any good places to eat around this area?",
    "Have you explored any of the historical sites in Manila?",
    "What's your favorite thing about living in Manila?",
    "Do you have any recommendations for things to do in this neighborhood?",
    "How do you usually cope with the heat here? Any tips?",
    "So Maria, tell me more about yourself.",
    "Finally, nice to meet you. so how did you get my contact?",
    "So Sam is my friend from college, how did you get to know him?",
    "Are you familiar with FWD?"
]

responses = []
deform_response = lambda format_response: format_response.split("<|start_header_id|>assistant<|end_header_id|>\n\n")[-1].split("<|eot_id|>")[0]
for prompt in prompts:
    format_response = f.generate(prompt)
    response = deform_response(format_response)
    responses.append(response)

In [ ]:
from vllm import SamplingParams, LLM

prompts = [
    "How is the wether like today?",
    "Do you have any insurance preference?",
    "Can I know your age?"
]

sampling_params = SamplingParams(
    temperature=0,
    max_tokens=256,
    stop=["<|eot_id|>"]
)
llm = LLM("Ksgk-fy/ecoach_philippine_v2_merge", params=sampling_params)

outputs = llm.generate(
    prompts,
    sampling_params,
)